# **A full 1D analysis using the low-level Gammapy API**

**Objective: Performing a full spectral anaysis of the point source [PKS 2155-304](http://tevcat.uchicago.edu/?mode=1&showsrc=90)**

Here we demonstrate the data reduction and spectral fitting for a point like source, using the [reflected regions background estimation](https://docs.gammapy.org/1.2/user-guide/makers/reflected.html?highlight=reflected) method.

In practice, we have to:
- Prepare the **data access and selection**
  - Create a `~gammapy.data.DataStore` poiting to the relevant data 
  - Apply an observation selection to produce a list of observations, a `~gammapy.data.Observations` object.
- Set up the **analyis parameters**
  - Define the [reconstructed energy](https://docs.gammapy.org/1.2/user-guide/references.html#term-Reco-Energy) axis and [true energy](https://docs.gammapy.org/1.2/user-guide/references.html#term-True-Energy) axis using the `~gammapy.maps.MapAxis` object
  - Define the spatial geometry
  - Define the [exclusion mask](https://docs.gammapy.org/1.2/tutorials/api/mask_maps.html)
  - Choose the correct `~gammapy.datasets.Dataset` type and define it
- Do the **data reduction**
  - Create the necessary makers : 
    - the spectrum dataset maker : `~gammapy.makers.SpectrumDatasetMaker`
    - the reflected regions finder: either `~gammapy.makers.ReflectedRegionsFinder` or `~gammapy.makers.WobbleRegionsFinder`
    - the background maker, here a `~gammapy.makers.ReflectedRegionsBackgroundMaker`
    - and usually the safe range maker : `~gammapy.makers.SafeMaskMaker`
  - Perform the data reduction loop. And for every observation:
    - Apply the makers sequentially to produce the current `~gammapy.datasets.SpectrumDatasetOnOff`
    - Stack it on the target one.
- Make **the modeling and fitting**
  - Define the `~gammapy.modeling.models.SkyModel` to fit the data. Being this a spectral analysis, the `SkyModel` is completely defined by a  `~gammapy.modeling.models.SpectralModel` (no spatial information is required)
  - Create a `~gammapy.modeling.Fit` object and run it to fit the model parameters
  - Apply a `~gammapy.estimators.FluxPointsEstimator` to compute flux points for the spectral part of the fit.

## Setup
First, we setup the analysis by performing required imports.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import logging
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from gammapy.maps import WcsGeom, MapAxis, RegionGeom
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    ExpCutoffPowerLawSpectralModel,
    PointSpatialModel,
    SkyModel,
    Models,
    FoVBackgroundModel,
    LogParabolaSpectralModel,
    EBLAbsorptionNormSpectralModel
)
from gammapy.makers import SafeMaskMaker, SpectrumDatasetMaker, ReflectedRegionsFinder, ReflectedRegionsBackgroundMaker
from gammapy.modeling import Fit
from gammapy.datasets import MapDataset, FluxPointsDataset, Datasets, SpectrumDataset
from scipy.stats import chi2
from gammapy.stats.utils import ts_to_sigma
from gammapy.catalog import SourceCatalog4FGL

from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion
from gammapy.estimators import FluxPointsEstimator
from gammapy.data import DataStore
from gammapy.visualization import plot_spectrum_datasets_off_regions

## Very optional set-up

As described in the documentation: the [HowTo](https://docs.gammapy.org/1.2/user-guide/howto.html) section, the search bar

**The logging level**

In [ ]:
logging.basicConfig()    
log = logging.getLogger("1Danalysis")
log.setLevel(logging.WARNING) #INFO, WARNING, DEBUG

**Suppression of the astropy warnings**

In [ ]:
# from astropy.io.fits.verify import VerifyWarning
# import warnings

# with warnings.catch_warnings():
#     warnings.simplefilter('ignore', VerifyWarning)

**Progress bar**

In [ ]:
# If the external library `tqdm` is installed
from gammapy.utils import pbar
pbar.SHOW_PROGRESS_BAR = True

## Defining the datastore and selecting observations

We first use the `~gammapy.data.DataStore` object to access the observations we want to analyse, here the H.E.S.S. DL3 DR1. 

In [ ]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1/")

In [ ]:
pos = SkyCoord(329.71693844, -30.22558846, unit=u.deg, frame="icrs")
pos.icrs

We can now define an observation filter to select only the relevant observations. 
Here we use a cone search that we define with a python dict.

We then filter the `ObservationTable` with `~gammapy.data.ObservationTable.select_sky_circle()`.

In [ ]:
obs_table_filtered = data_store.obs_table.select_sky_circle(center=pos, radius=2 * u.deg)
obs_ids = obs_table_filtered["OBS_ID"]

If you wish to apply more complex filtering options, you can use the `~gammapy.data.ObservationTable.select_observations()` method instead. This provides the freedom of selecting observations based on a sky circle, time period or parameter (e.g. Zenith angle) range.

We can now retrieve the relevant observations by passing their `obs_id` to the`~gammapy.data.DataStore.get_observations()` method.

In [ ]:
observations = data_store.get_observations(obs_ids)
print(f"Number of selected observations : {len(observations)}")

We restrict the analysis to the [July 2006 flaring event](https://ui.adsabs.harvard.edu/abs/2009A%26A...502..749A/abstract) using `gammapy.data.Observations.select_time()`.

In [ ]:
time_interval = Time(
    ["2006-07-29T20:30", "2006-07-30T20:30"]
)

In [ ]:
observations = observations.select_time(time_interval)
print(observations)

In [ ]:
obs = observations[12]
obs.events.peek()

In [ ]:
obs.aeff.peek();
obs.edisp.peek();
obs.psf.peek();
obs.bkg.peek();

## Preparing reduced datasets geometry

Now we define the [reconstructed](https://docs.gammapy.org/0.20/userguide/references.html#term-Reco-Energy) and [true](https://docs.gammapy.org/0.20/userguide/references.html#term-True-Energy) energy axes: 

In [ ]:
energy_axis = MapAxis.from_energy_bounds(
    0.1, 40, nbin=10, per_decade=True, unit="TeV", name="energy"
)
energy_axis_true = MapAxis.from_energy_bounds(
    0.05, 100, nbin=20, per_decade=True, unit="TeV", name="energy_true"
)

We then define a ON region to extract the spectrum, and create the analysis geometry using the `RegionGeom` object:

In [ ]:
on_region_radius = Angle("0.11 deg")
on_region = CircleSkyRegion(center=pos, radius=on_region_radius)

geom = RegionGeom.create(region=on_region, axes=[energy_axis])

Now we can define the dataset used for this 1D analysis using this geometry.

In [ ]:
dataset_empty = SpectrumDataset.create(
    geom=geom, energy_axis_true=energy_axis_true
)

## Create exclusion mask

To perform the spectral analysis we must mask all the gamma ray emission in the analysis region, which would otherwise bias the background estimation. Here we are analyzing an extra-Galactic source, which is isolated and would not require a priori an exclusion mask. However, for illustration purpose, we choose a mask of 0.5 deg to the North of the blazar.

In [ ]:
exclusion_region = CircleSkyRegion(
    center=SkyCoord(329.71, -29, unit="deg", frame="icrs"),
    radius=0.3 * u.deg,
)

skydir = pos.icrs
exgeom = WcsGeom.create(width=5*u.deg, binsz=0.02, skydir=skydir, proj="TAN", frame="icrs"
)

exclusion_mask = ~exgeom.region_mask([exclusion_region])
exclusion_mask.plot();

## Data reduction

### Create the maker classes to be used
We first initialize the `Maker` objects that will take care of the data reduction.

The `~gammapy.makers.SpectrumDatasetMaker`creates a `SpectrumDataset` for each observation.

In [ ]:
dataset_maker = SpectrumDatasetMaker(
    containment_correction=True, selection=["counts", "exposure", "edisp"]
)

**PS: zoom on the parameter `containment_correction'**

In the used data release, the IRFs are computed for a "full containment" response, ie all the signal for an objet falls into the analysis Region of Interest (there is no PSF leakage).

However, here for the 1D analysis with a `RegionGeom' of 0.11 deg, a part of the signal lies outside the selection region. Then, the IRFs, and mainly the collection area, should be corrected from the PSF leakage.

The `~gammapy.makers.ReflectedRegionsBackgroundMaker` appends a background estimate (based on the [reflected regions](https://docs.gammapy.org/1.0.1/user-guide/makers/reflected.html?highlight=reflected) method) to an input `SpectrumDataset`, converting it into a `SpectrumDatasetOnOff`.

In [ ]:
bkg_maker = ReflectedRegionsBackgroundMaker(exclusion_mask=exclusion_mask)

By default the `ReflectedRegionsBackgroundMaker` defines the OFF regions by rotating the ON region around the pointing position.

PS: if you need to apply more complex criteria to your OFF regions selection (e.g. an energy dependent rad-max cut / configure the number of OFF regions, etc) you can additionally pass to the `ReflectedRegionsBackgroundMaker` an instance of `~gammapy.makers.WobbleRegionsFinder` (see [this tutorial](https://docs.gammapy.org/1.2|/user-guide/makers/reflected.html)).

Finally we define a `~gammapy.makers.SafeMaskMaker` instance, which is responsible of selecting the safe data range (in energy and space) in which the data can be used.
In this example we only use the method `aeff-default`, which reads the safe energy threshold specified in the DL3 FITS files. For other available method see the documentation of the [SafeMaskMaker](https://docs.gammapy.org/1.0.1/api/gammapy.makers.SafeMaskMaker.html?highlight=safemaskmaker)

In [ ]:
safe_mask_masker = SafeMaskMaker(methods=["aeff-default"])
# safe_mask_masker = SafeMaskMaker(methods=["aeff-max"], aeff_percent=10)

### Perform the data reduction loop

For the moment the datasets are not stacked, but appended into a `Datasets` object (that basically contains a list of datasets). That's because we want to produce diagnostic plots such as the cumulative source significance as a function of the observation livetime. The stacking will be performed later on. 

In [ ]:
%%time

datasets = Datasets()
for observation in observations:
    # First a spectrum dataset with the same geometry as the reference empty one is filled with 
    # the data and IRFs
    dataset = dataset_maker.run(dataset_empty.copy(name=str(observation.obs_id)), observation)
    # Reflected regions background estimation
    dataset_on_off = bkg_maker.run(dataset, observation)
    # The data quality cut is applied
    dataset_on_off = safe_mask_masker.run(dataset_on_off, observation)
    # The resulting dataset is appended to the list
    datasets.append(dataset_on_off)

In [ ]:
ax = exclusion_mask.plot();
plot_spectrum_datasets_off_regions(ax=ax, datasets=datasets);
on_region.to_pixel(ax.wcs).plot(ax=ax, color="black");

In [ ]:
info_table = datasets.info_table(cumulative=True)
info_table
# info_table[:5]
# display(info_table)

In [ ]:
plt.figure(figsize=(10, 10))
ax1 = plt.subplot(221)
ax1.plot(
    info_table["livetime"].to("h"), info_table["excess"], marker="o", ls="none"
)
ax1.set_xlabel("Livetime [h]")
ax1.set_ylabel("Excess");


ax2 = plt.subplot(222)
ax2.plot(
    info_table["livetime"].to("h"),
    info_table["sqrt_ts"],
    marker="o",
    ls="none",
)
ax2.set_xlabel("Livetime [h]")
ax2.set_ylabel("Sqrt(TS)");


Save the dataset to disc using `~gammapy.datasets.Datasets.write()` method.

PS: The OGIP format is used (PHA, ARF, RMF, BKG, [see](https://gamma-astro-data-formats.readthedocs.io/en/latest/spectra/ogip/index.html) here for details)

In [ ]:
filename = "pks-joint-dataset.yaml"
datasets.write(filename, overwrite=True)

In [ ]:
#! cat pks-joint-dataset.yaml

Before fitting we stack the `Datasets` into a single `SpectrumDatasetOnOff`:

In [ ]:
#create stacked
stacked = datasets.stack_reduce()

In [ ]:
print(stacked)

In [ ]:
stacked.peek()

## Data Fitting

In this section we fit a spectral model to the data. We can try to answer the following questions:
- What is the significance of the detected source?
- What is the best spectral shape to describe the spectrum of the source? 

In particular, we can use the [likelihood ratio test](https://docs.gammapy.org/1.2/user-guide/stats/index.html#estimating-ts) (see also the associated HowTo) to compare three different hypotheses:
- H0: Background only (no source)
- H1: Background + source described by a power law model
- H2: Background + source described by a power law model with exponential cutoff

### **H0**

The value of the quantity $-2\ln\mathcal(L)$ for the background-only model (null hypothesis) can be simply computed as

In [ ]:
Wstat_0 = stacked.stat_sum()
print(Wstat_0)

Since the background has been estimated using the reflected regions method, here $-2\ln\mathcal(L)$ corresponds to the so-called [Wstat](https://docs.gammapy.org/0.20/userguide/references.html#term-WStat) fit statistic.

We can inspect the model residuals for the H0 hypothesis:

In [ ]:
stacked.plot_residuals_spectral();

As expected, the residuals show a clear positive feature indicating that a source is missing in the model. 

## **H1**
We now add a source defined by a power law spectrum to the model.

Here we also consider [EBL absorption](https://docs.gammapy.org/1.2/user-guide/model-gallery/spectral/plot_absorbed.html?highlight=ebl). To get the list of EBL models, you can type:

In [ ]:
# import gammapy.modeling.models.spectral as spectral
# spectral.EBL_DATA_BUILTIN

In [ ]:
spectral_model_pl = PowerLawSpectralModel()
redshift = 0.116
ebl = EBLAbsorptionNormSpectralModel.read_builtin("finke", redshift=redshift)
spectral_model_1 = spectral_model_pl * ebl

pks_model_1 = SkyModel(spectral_model=spectral_model_1,
                    name="pks_model")

In [ ]:
pks_model_1.parameters.to_table()

In [ ]:
stacked.models = [pks_model_1]

In [ ]:
%%time
fit1 = Fit(optimize_opts={"print_level": 1})
result1 = fit1.run(datasets=[stacked])

In [ ]:
result1.success
# print(result1)

In [ ]:
result1.models.to_parameters_table()

In [ ]:
Wstat_1 = result1.total_stat
delta_ts = Wstat_0-Wstat_1
df = len(result1.models.parameters.free_parameters.names)
sigma = ts_to_sigma(delta_ts, df=df)
print(f"The delta_ts  of H1 vs H0: {delta_ts:.3f}, that gives a p-value of {chi2.sf(delta_ts, df)}")
print(f"Converting this to a significance gives: {sigma:.3f} \u03C3")

In [ ]:
ax_spectrum, ax_residuals = stacked.plot_fit()
# ax_spectrum.set_xlim(0.2, 70)
# stacked.plot_masks(ax=ax_spectrum);

We can compute flux points for the H1 model assumption.

In [ ]:
energy_edges = np.logspace(-1, 1.6, 12)*u.TeV
fpe = FluxPointsEstimator(energy_edges=energy_edges, source=pks_model_1.name, selection_optional=["ul"])

In [ ]:
%%time
flux_points = fpe.run(datasets=[stacked])

In [ ]:
ax = spectral_model_pl.plot(energy_bounds=[0.1,20]*u.TeV, sed_type="e2dnde", label="intrinsic", color="blue")
spectral_model_1.plot(ax=ax, energy_bounds=[0.1,20]*u.TeV, sed_type="e2dnde", label="absorbed", color="red")
spectral_model_1.plot_error(ax=ax, energy_bounds=[0.1,20]*u.TeV, sed_type="e2dnde", facecolor="red")
flux_points.plot(ax=ax, sed_type="e2dnde")
plt.legend();

In [ ]:
spectral_model_1.to_dict()

## **H2**

We now estimate the significance for the presence of an exponential cutoff in the source spectrum, again taking into account the EBL absorption.

In [ ]:
spectral_model_ecpl = ExpCutoffPowerLawSpectralModel()
redshift = 0.116
spectral_model_2 = spectral_model_ecpl * ebl

pks_model_2 = SkyModel(spectral_model=spectral_model_2,
                    name="pks_model")

In [ ]:
pks_model_2.parameters.to_table()

In [ ]:
stacked.models = [pks_model_2]

In [ ]:
%%time
fit2 = Fit(optimize_opts={"print_level": 1})
result2 = fit2.run(datasets=[stacked])

In [ ]:
result2.success

In [ ]:
result2.models.to_parameters_table()

In [ ]:
Wstat_2 = result2.total_stat
delta_ts = Wstat_1-Wstat_2
df = len(result2.models.parameters.free_parameters.names)-len(result1.models.parameters.free_parameters.names)
sigma = ts_to_sigma(delta_ts, df=df)
print(f"The delta_ts  of H2 vs H1: {delta_ts:.3f}, that gives a p-value of {chi2.sf(delta_ts, df)}")
print(f"Converting this to a significance gives: {sigma:.3f} \u03C3")

We have successfully detected a cutoff in the observed spectrum.

In [ ]:
ax_spectrum, ax_residuals = stacked.plot_fit()

To check that the fit has coverged correctly, it is always a good idea to inspect the likelihood profile for the free model parameters.

In [ ]:
total_stat = result2.total_stat

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(14, 4))

for ax, par in zip(axes, pks_model_2.parameters.free_parameters):
    par.scan_n_values = 25

    profile = fit2.stat_profile(datasets=[stacked], parameter=par)
    ax.plot(profile[f"{pks_model_2.name}.spectral.{par.name}_scan"], profile["stat_scan"] - total_stat)
    ax.set_xlabel(f"{par.unit}")
    ax.set_ylabel("Delta TS")
    ax.set_title(f"{par.name}: {par.value:.1e} +- {par.error:.1e}")

We now compute the flux points for the H2 hypothesis.

In [ ]:
%%time
flux_points = fpe.run(datasets=[stacked])

In [ ]:
ax = spectral_model_ecpl.plot(energy_bounds=[0.1,20]*u.TeV, sed_type="e2dnde", label="intrinsic", color="blue")
spectral_model_2.plot(ax=ax, energy_bounds=[0.1,20]*u.TeV, sed_type="e2dnde", label="absorbed", color="red")
spectral_model_2.plot_error(ax=ax, energy_bounds=[0.1,20]*u.TeV, sed_type="e2dnde", facecolor="red")
flux_points.plot(ax=ax, sed_type="e2dnde")
plt.legend();

In [ ]:
flux_points.to_table()

# Exercises:

## Beginner
- Select and analyze observations of PSK 2155-304 during its steady state 
- Try other models, eg: log-parabola, broken power law, etc. See the model gallery for a list of available models: https://docs.gammapy.org/1.2/user-guide/model-gallery/index.html
- What is the impact of changing the OFF regions criteria (their number, shape, finding method)? 
- Try to repeat the fit using a different minimizer. By default Gammapy uses Minuit, but it also supports the Sherpa and Scipy backends.

## Advanced
- Create a gammapy.estimators.FluxPointsDataset with the flux points you have computed for the stacked dataset and fit the flux points again with one of the spectral models. How does the result compare to the best fit model, that was directly fitted to the counts data?
- Compute a 2-dimensional likelihood contour to estimate the correlation between the fitted parameters (e.g. the spectral index and cutoff). (Tutorial reference: https://docs.gammapy.org/1.2/tutorials/api/fitting.html)
- Repeat exercise on the Crab runs available in GAMMAPY_DATA. Alternatively, if you have access to CTA DC1 simulated data, repeat on your favourite source 